In [49]:
from datetime import datetime, timedelta
import requests
from datetime import datetime
import pandas as pd

In [50]:
api_key = "70fe7264b5b42357e418290a7f8ff9c6"
username = "hangisack"

In [51]:
class LastFMAPI:
    WEEK_DIFF = 604800
    def __init__(self, username, key, limit=1000):
        self.username = username
        self.key = key
        self.limit = limit
        self.start_date = datetime(year=2024, month=2, day=12)  # first time using lastfm
        self.start_utc = int(self.start_date.timestamp()) # utc
        self.dfs = []
        self.current_time = datetime.now()
        self.utc_time = int(self.current_time.timestamp())
        self.df = pd.DataFrame()
        
    def get_start_date(self):
        return self.start_utc
    
    def set_next_start_date(self):
        if self.start_utc + LastFMAPI.WEEK_DIFF > self.utc_time:
            self.start_utc = self.utc_time
        else:
            self.start_utc += LastFMAPI.WEEK_DIFF
        
    def get_end_date(self):
        return self.start_utc + LastFMAPI.WEEK_DIFF
    
    def get_tags(self, artists):  # only alphanumeric
        tags = {}
        for artist in (set(artists)):
            url = f"https://ws.audioscrobbler.com/2.0/?method=artist.getTopTags&artist={artist}&api_key={self.key}&format=json"
            res = requests.get(url).json()
            top_5 = [tag['name'] for tag in res['toptags']['tag'][:5]]
            tags[artist] = top_5
        return tags
            
        
        
    def get_and_save_all_tags(self):
        self.df['tags'] = self.get_tags_vectorized(self.df['artist'])
        self.df.to_csv("alltracks.csv", encoding="utf-8")

    def get_tags_vectorized(self, artists):
        tags_dict = self.get_tags(artists)
        return artists.map(lambda artist: ', '.join(tags_dict.get(artist, [])))

        
    def save_tracks(self):
        url = f"https://ws.audioscrobbler.com/2.0/?api_key={self.key}&method=user.getrecenttracks&user={self.username}&from={self.get_start_date()}&to={self.get_end_date()}&limit={self.limit}&format=json"
        res = requests.get(url).json()
        tracks = res['recenttracks']["track"]
        track_name = []
        artist = []
        album_name = []
        date_listened = []
        uts = []
        urls = []
        for track in tracks:  # might be more efficient to build the dataframe by row, instead of column.
            try:
                artist.append(track['artist']['#text'])
            except KeyError:
                artist.append("NA")
            
            try:
                album_name.append(track['album']['#text'])
            except KeyError:
                album_name.append("NA")
            
            try:
                track_name.append(track['name'])
            except KeyError:
                track_name.append("NA")
            
            try:
                date_listened.append(track['date']['#text'])
            except KeyError:
                date_listened.append("NA")
            
            try:
                uts.append(track['date']['uts'])
            except KeyError:
                uts.append("NA")
            
            try:
                urls.append(track['url'])
            except KeyError:
                urls.append("NA")
            
        df = pd.DataFrame({
            'track_name': track_name,
            'artist': artist,
            'album_name': album_name,
            'date_listened': date_listened,
            'uts': uts,
            'url': urls,
        })
        self.dfs.append(df)
    def get_csv_file(self):
        merged_table = pd.concat(self.dfs, ignore_index=True)
        self.df = merged_table
        merged_table.to_csv("alltracks.csv", encoding="utf-8")
    
    def get_all_tracks(self):
        while self.get_start_date() < self.utc_time:
            self.save_tracks()
            self.set_next_start_date() 
            print("Saving..")
        print("DONE!")
        

In [52]:
lastfmapi = LastFMAPI(username, api_key)
lastfmapi.get_all_tracks()
lastfmapi.get_csv_file()


Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
DONE!


In [53]:
lastfmapi.get_and_save_all_tags()


In [54]:
df = pd.read_csv("alltracks.csv")
df.head()

,Unnamed: 0,track_name,artist,album_name,date_listened,uts,url,tags
0,0,No Class,Motörhead,Overkill,NaN,NaN,https://www.last.fm/music/Mot%C3%B6rhead/_/No+...,"heavy metal, hard rock, metal, speed metal, se..."
1,1,2122,Geese,3D Country,"18 Feb 2024, 10:58",1.708254e+09,https://www.last.fm/music/Geese/_/2122,"indie rock, post-punk, art punk, Crank wave, d..."
2,2,Rodeo in Joliet,The Jesus Lizard,Goat (Remaster / Reissue),"18 Feb 2024, 10:53",1.708254e+09,https://www.last.fm/music/The+Jesus+Lizard/_/R...,"noise rock, Grunge, alternative, noise, rock"
3,3,Lady Shoes,The Jesus Lizard,Goat (Remaster / Reissue),"18 Feb 2024, 10:50",1.708253e+09,https://www.last.fm/music/The+Jesus+Lizard/_/L...,"noise rock, Grunge, alternative, noise, rock"
4,4,South Mouth,The Jesus Lizard,Goat (Remaster / Reissue),"18 Feb 2024, 10:47",1.708253e+09,https://www.last.fm/music/The+Jesus+Lizard/_/S...,"noise rock, Grunge, alternative, noise, rock"


In [55]:
df.tail()

,Unnamed: 0,track_name,artist,album_name,date_listened,uts,url,tags
10486,10486,Capricorn,Motörhead,Overkill,"22 Jul 2024, 00:36",1.721609e+09,https://www.last.fm/music/Mot%C3%B6rhead/_/Cap...,"heavy metal, hard rock, metal, speed metal, se..."
10487,10487,I'll Be Your Sister,Motörhead,Overkill,"22 Jul 2024, 00:33",1.721608e+09,https://www.last.fm/music/Mot%C3%B6rhead/_/I%2...,"heavy metal, hard rock, metal, speed metal, se..."
10488,10488,(I Won't) Pay Your Price,Motörhead,Overkill,"22 Jul 2024, 00:30",1.721608e+09,https://www.last.fm/music/Mot%C3%B6rhead/_/(I+...,"heavy metal, hard rock, metal, speed metal, se..."
10489,10489,Stay Clean,Motörhead,Overkill,"22 Jul 2024, 00:27",1.721608e+09,https://www.last.fm/music/Mot%C3%B6rhead/_/Sta...,"heavy metal, hard rock, metal, speed metal, se..."
10490,10490,Overkill,Motörhead,Overkill,"22 Jul 2024, 00:22",1.721608e+09,https://www.last.fm/music/Mot%C3%B6rhead/_/Ove...,"heavy metal, hard rock, metal, speed metal, se..."
